In [28]:
import numpy as np

from scipy import ndimage

https://adventofcode.com/2020

# Day 17

In [5]:
input17 = """..##.#.#
.#####..
#.....##
##.##.#.
..#...#.
.#..##..
.#...#.#
#..##.##"""

In [51]:
def map_to_array(astmap):
    arr2d = np.array([[e=='#' for e in row] for row in astmap.split('\n')]).T
    return arr2d.copy()

def array_to_map(arr, marker='#'):
    buffer = []
    for row in arr.T:
        for element in row:
            buffer.append(marker if element else '.')
        buffer.append('\n')
    return ''.join(buffer)

## Part 1

In [53]:
test_input = """.#.
..#
###"""
testarr = map_to_array(test_input)
print(array_to_map(testarr))

.#.
..#
###



In [65]:
boot_field = np.zeros((testarr.shape[0] + 6*2, testarr.shape[1] + 6*2, 1 + 6*2), dtype=bool)
boot_field[6:9, 6:9, 6] = testarr
print(array_to_map(boot_field[..., 6]))

...............
...............
...............
...............
...............
...............
.......#.......
........#......
......###......
...............
...............
...............
...............
...............
...............



In [101]:
neighbor_kernel = [[[1,1,1],
                    [1,1,1],
                    [1,1,1]],
                   [[1,1,1],
                    [1,0,1],
                    [1,1,1]],
                   [[1,1,1],
                    [1,1,1],
                    [1,1,1]]]
neighbor_kernel = np.array(neighbor_kernel)
assert np.sum(neighbor_kernel) == 26

def do_step(initial_field, nsteps=1):
    if nsteps==0:
        return initial_field.reshape(initial_field.shape[0], initial_field.shape[1], 1)
    
    boot_field = np.zeros((initial_field.shape[0] + nsteps*2, 
                           initial_field.shape[1] + nsteps*2, 
                           1 + nsteps*2), dtype=int)
    boot_field[nsteps:(nsteps + initial_field.shape[0]), 
               nsteps:(nsteps + initial_field.shape[1]), 
               nsteps] = initial_field.astype(int)
    
    for i in range(nsteps):
        corr = ndimage.correlate(boot_field, neighbor_kernel, mode='constant')
        new_field = boot_field & (corr==2)
        new_field[corr==3] = True
        boot_field = new_field.astype(int)
    
    return boot_field

step3 = do_step(testarr, nsteps=3)
                     
                     
assert array_to_map(step3[1:-1, 2:, 3]) == """...#...
.......
#......
.......
.....##
.##.#..
...#...
"""

assert np.sum(do_step(testarr, nsteps=6)) == 112

In [102]:
np.sum(do_step(map_to_array(input17), nsteps=6))

213

## Part 2 

In [105]:
neighbor_kernel4 = np.ones((3, 3, 3, 3), dtype=int)
neighbor_kernel4[1,1,1,1] = 0
neighbor_kernel4 = np.array(neighbor_kernel4)
assert np.sum(neighbor_kernel4) == 80

In [117]:
def do_step4(initial_field, nsteps=1):
    if nsteps==0:
        return initial_field.reshape(initial_field.shape[0], initial_field.shape[1], 1, 1)
    
    boot_field = np.zeros((initial_field.shape[0] + nsteps*2, 
                           initial_field.shape[1] + nsteps*2, 
                           1 + nsteps*2, 1 + nsteps*2), dtype=int)
    boot_field[nsteps:(nsteps + initial_field.shape[0]), 
               nsteps:(nsteps + initial_field.shape[1]), 
               nsteps, nsteps] = initial_field.astype(int)
    
    for i in range(nsteps):
        corr = ndimage.correlate(boot_field, neighbor_kernel4, mode='constant')
        new_field = boot_field & (corr==2)
        new_field[corr==3] = True
        boot_field = new_field.astype(int)
    
    return boot_field

assert array_to_map(do_step4(testarr, 2)[1:-1, 2:, 2, 0]) == """###..
##.##
#...#
.#..#
.###.
""""""

In [131]:
assert np.sum(do_step4(testarr, nsteps=6)) == 848

In [132]:
np.sum(do_step4(map_to_array(input17), nsteps=6))

1624

# Day N

In [ ]:
!mv ~/Desktop/input.txt inputN

In [ ]:
with open('inputN') as f:
    inputN = f.read().strip()

## Part 1

## Part 2 